In [1]:
import pandas as pd
import os
import codecs
import shutil
import datetime
import time

In [2]:
def get_standard_account(corp_code, account_type): # 회사별 표준계정 탬플릿 불러오기!
    if account_type == "포괄손익계산서":
        eng_type = "income_statement"
    elif account_type == "재무상태표":
        eng_type = "balance_sheet"
    elif account_type == "현금흐름표":
        eng_type = "cash_flow"
    path = os.path.join("dart", "dart", corp_code, "financial_statements", "standard_account.xlsx")
    df = pd.read_excel(path, engine='openpyxl', sheet_name = account_type)
    return df

In [3]:
def get_db_files(corp_code, account_type): # 해당 회사 분기 손익계산서 / 재무상태표 / 현금흐름표 불러오기
    if account_type == "포괄손익계산서":
        eng_type = "income_statement"
    elif account_type == "재무상태표":
        eng_type = "balance_sheet"
    elif account_type == "현금흐름표":
        eng_type = "cash_flow"
    path = os.path.join("dart", "dart", corp_code, "financial_statements", eng_type)
    files = os.listdir(path)
    result = []
    for file in files:
        if file[0].isdigit(): # 분기보고서만 add.
            result.append(file) 
    return result

#get_db_files("005810", "재무상태표")

In [4]:
def get_corp_db_quarters(corp_code, account_type): # 해당 회사의 DB에 저장된 파일들을 표준계정의 column과 동일하게 하는 작업.
    if account_type == "포괄손익계산서":
        eng_type = "income_statement"
    elif account_type == "재무상태표":
        eng_type = "balance_sheet"
    elif account_type == "현금흐름표":
        eng_type = "cash_flow"
    path = os.path.join("dart", "dart", corp_code, "financial_statements", eng_type)
    files = get_db_files(corp_code, account_type)
    columns = []
    for file in files:
        if "_year_" in file:
            year = str(int(file.split("_")[-1].split(".csv")[0][:4]) -1)
            #print(year)
        else:
            year = file.split("_")[-1].split(".csv")[0][:4]
        quarter = file.split("_")[1]

        if quarter == "half year":
            quarter = "q2"
        elif quarter == "year":
            quarter = "q4"

        column = year + "." + quarter + "(항목)"
        columns.append(column)
    return columns

#get_corp_db_quarters("005930", "재무상태표")

In [5]:
def create_corp_income_dict(corp_code, file, account_type): # 해당 회사의 특정 분기 파일 데이터를 딕셔너리화 하는 작업.
    if account_type == "포괄손익계산서":
        eng_type = "income_statement"
    elif account_type == "재무상태표":
        eng_type = "balance_sheet"
    elif account_type == "현금흐름표":
        eng_type = "cash_flow"
        
    s = os.path.join("dart", "dart", corp_code, "financial_statements", eng_type, file)
    f = codecs.open(s, "r", "euc-kr")

    data = f.readlines() # 메인 파일에서 인코딩이 ut-8로 되어있기 때문에 이렇게 해줘야 함.

    column = []
    value = []
    for d in data:
        d = d.split(",")
        column.append(d[0])
        clean_value = d[1].split('\r')[0]
        value.append(clean_value)
    column.pop(0)
    value.pop(0)

    k = column
    v = value
    corp_income_dict = {}
    for i in range(len(v)):
        #print(v[i])
        if "E+" in v[i]:
            raw = float(v[i].split("E+")[0])
            #print(raw)
            power = int(v[i].split("E+")[-1])
            #print(power)
            corp_income_dict[k[i]] = raw * (10 ** power)
            continue
        corp_income_dict[k[i]] = v[i]
    return corp_income_dict

#create_corp_income_dict("005930", "49_q1_c_balance_sheet_20170515.csv", "재무상태표")

In [6]:
def get_entry_iloc(corp_code, file, account_type):
    if account_type == "포괄손익계산서":
        eng_type = "income_statement"
    elif account_type == "재무상태표":
        eng_type = "balance_sheet"
    elif account_type == "현금흐름표":
        eng_type = "cash_flow"
    path = os.path.join("dart", "dart", corp_code, "financial_statements", eng_type, file)
    f = codecs.open(path, "r", "euc-kr")
    data = f.readlines()
    column_dict = {}
    i = 0
    for d in data:
        d = d.split(",")
        column_dict[d[0]] = i
        i += 1
    return column_dict
#get_entry_iloc("005880", "51_q1_c_income_statement_20170515.csv")

In [7]:
def is_std_account_exist(corp_code): # 회사 폴더 안에 표준계정 탬플릿 있는지 확인하고, 없으면 생성.
    path = os.path.join("dart", "dart", corp_code, "financial_statements")
    if "standard_account.xlsx" in os.listdir(path):
        return True
    else:
        return False
#is_std_account_exist("005810")

In [8]:
def update_corp_income(corp_code):
    # 표준계정 항목들의 인덱스 모음.
    std_account_entry_iloc = {'매출액(수익)' : 0, '매출원가' : 14, '매출총이익': 25, '판매비와관리비' : 26, '영업이익' : 57, '기타영업손익' : 58, '영업이익(발표기준)' : 98, '[구K-IFRS]영업이익' : 99, '금융수익' : 100, '금융원가' : 125, '기타영업외손익' : 149, '종속기업,공동지배기업및관계기업관련손익' : 196, '당기순이익' : 216, '기타포괄이익' : 220, '총포괄이익' : 232}
    
    if not is_std_account_exist(corp_code): # 표준계정 없으면, 자동 생성.
        original = 'template.xlsx'
        target = os.path.join("dart", "dart", corp_code, "financial_statements", "standard_account.xlsx")
        shutil.copyfile(original, target)
        print(corp_code, "표준계정 생성!")
        
    df = get_standard_account(corp_code, "포괄손익계산서") # 해당 회사 표준계정 불러오기.
    files = get_db_files(corp_code, "포괄손익계산서") # 해당 회사 DB의 전체 파일들 불러오기
    quarters = get_corp_db_quarters(corp_code, "포괄손익계산서") # 파일들의 해당 분기별 리스트 데이터
    for i in range(len(files)):
        corp_income_dict = create_corp_income_dict(corp_code, files[i], "포괄손익계산서") # 각 분기별 파일들의 딕셔너리 데이터.
        entry_iloc = get_entry_iloc(corp_code, files[i], "포괄손익계산서") # 각 파일 항목들의 인덱스.

        for k in corp_income_dict: # 표준계정에 해당 데이터 입력
            # k == 분기 보고서 파일에 있는 항목 이름.
            if k == '당기순이익(손실)':
                df.loc[std_account_entry_iloc['당기순이익'],quarters[i]] = corp_income_dict['당기순이익(손실)'] # corp_income_dict[k] 로 해도 무방함.
                print(corp_code, "당기순이익", quarters[i], "표준계정에 업데이트 완료!")
            if k == '당기순이익':
                df.loc[std_account_entry_iloc['당기순이익'],quarters[i]] = corp_income_dict['당기순이익']
                print(corp_code, "당기순이익", quarters[i], "표준계정에 업데이트 완료!")
            elif k == '세후기타포괄손익':
                df.loc[std_account_entry_iloc['기타포괄이익'],quarters[i]] = corp_income_dict['세후기타포괄손익']
                print(corp_code, "기타포괄손익", quarters[i], "표준계정에 업데이트 완료!")
            elif k == '기타포괄손익':
                df.loc[std_account_entry_iloc['기타포괄이익'],quarters[i]] = corp_income_dict['기타포괄손익']
                print(corp_code, "기타포괄손익", quarters[i], "표준계정에 업데이트 완료!")
            elif k == '총포괄손익':
                df.loc[std_account_entry_iloc['총포괄이익'],quarters[i]] = corp_income_dict['총포괄손익']
                print(corp_code, "총포괄손익", quarters[i], "표준계정에 업데이트 완료!")
            elif k == '당기총포괄이익':
                df.loc[std_account_entry_iloc['총포괄이익'],quarters[i]] = corp_income_dict['당기총포괄이익']
                print(corp_code, "총포괄손익", quarters[i], "표준계정에 업데이트 완료!")
            elif k == '매출액':
                df.loc[std_account_entry_iloc['매출액(수익)'],quarters[i]] = corp_income_dict['매출액']
                print(corp_code, "매출액", quarters[i], "표준계정에 업데이트 완료!")
            elif k == '수익(매출액)':
                df.loc[std_account_entry_iloc['매출액(수익)'],quarters[i]] = corp_income_dict['수익(매출액)']
                print(corp_code, "매출액", quarters[i], "표준계정에 업데이트 완료!")
            elif k == '매출원가':
                df.loc[std_account_entry_iloc['매출원가'],quarters[i]] = corp_income_dict['매출원가']
                print(corp_code, "매출원가", quarters[i], "표준계정에 업데이트 완료!")
            elif k == '매출총이익':
                df.loc[std_account_entry_iloc['매출총이익'],quarters[i]] = corp_income_dict['매출총이익']
                print(corp_code, "매출총이익", quarters[i], "표준계정에 업데이트 완료!")
            elif k == '판매비와관리비':
                df.loc[std_account_entry_iloc['판매비와관리비'],quarters[i]] = corp_income_dict['판매비와관리비']
                print(corp_code, "판매비와관리비", quarters[i], "표준계정에 업데이트 완료!")
            elif k == '영업이익(손실)':
                df.loc[std_account_entry_iloc['영업이익'],quarters[i]] = corp_income_dict['영업이익(손실)']
                print(corp_code, "영업이익", quarters[i], "표준계정에 업데이트 완료!")
            elif k == '영업이익':
                df.loc[std_account_entry_iloc['영업이익'],quarters[i]] = corp_income_dict['영업이익']
                print(corp_code, "영업이익", quarters[i], "표준계정에 업데이트 완료!")
            elif k == '금융수익':
                df.loc[std_account_entry_iloc['금융수익'],quarters[i]] = corp_income_dict['금융수익']
                print(corp_code, "금융수익", quarters[i], "표준계정에 업데이트 완료!")
            elif k == '금융원가':
                df.loc[std_account_entry_iloc['금융원가'],quarters[i]] = corp_income_dict['금융원가']
                print(corp_code, "금융원가", quarters[i], "표준계정에 업데이트 완료!")
            
            # 일치하지 않는 항목 업데이트 리스트 => But, 시간 너무 오래 잡아먹음.
            else:
                with open('corp_income_dummy.txt', 'a', encoding="euc-kr") as f:
                    f.write(k)
                    f.write('\n')
                print("일치하지 않는 포괄손익계산서 항목이름", k, "가 더미파일에 추가되엇습니다.")
            
                
    path = os.path.join("dart", "dart", corp_code, "financial_statements", "standard_account.xlsx")
    
    df1 = get_standard_account(corp_code, "재무상태표")
    df2 = get_standard_account(corp_code, "현금흐름표")
    with pd.ExcelWriter(path) as writer:
        df.to_excel(writer, encoding="euc-kr", sheet_name = "포괄손익계산서", index = False) # 저장
        df1.to_excel(writer, sheet_name="재무상태표", index = False)
        df2.to_excel(writer, sheet_name="현금흐름표", index = False)
    #print(df)
    
#update_corp_income("000020")

000020 매출액 2017.q2(항목) 표준계정에 업데이트 완료!
000020 매출원가 2017.q2(항목) 표준계정에 업데이트 완료!
000020 매출총이익 2017.q2(항목) 표준계정에 업데이트 완료!
000020 판매비와관리비 2017.q2(항목) 표준계정에 업데이트 완료!
일치하지 않는 포괄손익계산서 항목이름 연구개발비 가 더미파일에 추가되엇습니다.
000020 영업이익 2017.q2(항목) 표준계정에 업데이트 완료!
일치하지 않는 포괄손익계산서 항목이름 기타수익 가 더미파일에 추가되엇습니다.
일치하지 않는 포괄손익계산서 항목이름 기타비용 가 더미파일에 추가되엇습니다.
일치하지 않는 포괄손익계산서 항목이름 금융 수익 가 더미파일에 추가되엇습니다.
일치하지 않는 포괄손익계산서 항목이름 금융 비용 가 더미파일에 추가되엇습니다.
일치하지 않는 포괄손익계산서 항목이름 지분법손익 가 더미파일에 추가되엇습니다.
일치하지 않는 포괄손익계산서 항목이름 법인세비용차감전순이익(손실) 가 더미파일에 추가되엇습니다.
일치하지 않는 포괄손익계산서 항목이름 법인세비용 가 더미파일에 추가되엇습니다.
000020 당기순이익 2017.q2(항목) 표준계정에 업데이트 완료!
일치하지 않는 포괄손익계산서 항목이름 당기순이익(손실) 가 더미파일에 추가되엇습니다.
000020 기타포괄손익 2017.q2(항목) 표준계정에 업데이트 완료!
일치하지 않는 포괄손익계산서 항목이름 후속적으로 당기손익으로 재분류되는 포괄손익 가 더미파일에 추가되엇습니다.
일치하지 않는 포괄손익계산서 항목이름 매도가능증권평가이익 가 더미파일에 추가되엇습니다.
일치하지 않는 포괄손익계산서 항목이름 지분법자본변동 가 더미파일에 추가되엇습니다.
000020 총포괄손익 2017.q2(항목) 표준계정에 업데이트 완료!
일치하지 않는 포괄손익계산서 항목이름 기본주당이익(손실) 가 더미파일에 추가되엇습니다.
일치하지 않는 포괄손익계산서 항목이름 희석주당이익(손실) 가 더미파일에 추가되엇습니다.
000020 매출액 2017.q1

000020 매출액 2020.q3(항목) 표준계정에 업데이트 완료!
000020 매출원가 2020.q3(항목) 표준계정에 업데이트 완료!
000020 매출총이익 2020.q3(항목) 표준계정에 업데이트 완료!
000020 판매비와관리비 2020.q3(항목) 표준계정에 업데이트 완료!
일치하지 않는 포괄손익계산서 항목이름 연구개발비 가 더미파일에 추가되엇습니다.
000020 영업이익 2020.q3(항목) 표준계정에 업데이트 완료!
일치하지 않는 포괄손익계산서 항목이름 기타이익 가 더미파일에 추가되엇습니다.
일치하지 않는 포괄손익계산서 항목이름 기타손실 가 더미파일에 추가되엇습니다.
000020 금융수익 2020.q3(항목) 표준계정에 업데이트 완료!
000020 금융원가 2020.q3(항목) 표준계정에 업데이트 완료!
일치하지 않는 포괄손익계산서 항목이름 지분법손익 가 더미파일에 추가되엇습니다.
일치하지 않는 포괄손익계산서 항목이름 법인세비용차감전순이익(손실) 가 더미파일에 추가되엇습니다.
일치하지 않는 포괄손익계산서 항목이름 법인세비용 가 더미파일에 추가되엇습니다.
000020 당기순이익 2020.q3(항목) 표준계정에 업데이트 완료!
일치하지 않는 포괄손익계산서 항목이름 당기순이익(손실) 가 더미파일에 추가되엇습니다.
000020 기타포괄손익 2020.q3(항목) 표준계정에 업데이트 완료!
일치하지 않는 포괄손익계산서 항목이름 당기손익으로 재분류되지 않는항목(세후기타포괄손익) 가 더미파일에 추가되엇습니다.
일치하지 않는 포괄손익계산서 항목이름 기타포손익-공정가치측정지분상품평가손익 가 더미파일에 추가되엇습니다.
일치하지 않는 포괄손익계산서 항목이름 당기손익으로 재분류될 수 있는 항목(세후기타포괄손익) 가 더미파일에 추가되엇습니다.
일치하지 않는 포괄손익계산서 항목이름 지분법자본변동 가 더미파일에 추가되엇습니다.
000020 총포괄손익 2020.q3(항목) 표준계정에 업데이트 완료!
일치하지 않는 포괄손익계산서 항목이름 지배기업 소유주지분 가 더미파일에 추가되엇

In [9]:
def update_balance_sheet(corp_code):
    # 표준계정 항목들의 인덱스 모음.
    std_account_entry_iloc = {'자산총계' : 0, '유동자산' : 1, '재고자산' : 2, '단기금융자산' : 20, '매출채권및기타채권' : 30, 
                              '기타유동자산' : 41, '매각예정비유동자산및처분자산' : 47, '유형자산' : 52, '무형자산' :72, 
                              '투자부동산' : 85, '투자자산' : 89, '장기매출채권및기타채권' : 107, '기타비유동자산' : 118, 
                              '유동성장기부채' :130, '유동금융부채' : 140, '매입채무및기타채무' : 149, '기타유동부채' : 164, 
                              '매각예정으로분류된처분자산에포함된부채' : 170, '사채' : 175, '비유동금융부채' : 183, 
                              '장기매입채무및기타채무' : 190, '기타비유동부채' : 206, '자본금' : 217, '자본잉여금' : 221, 
                              '기타자본' : 227, '기타포괄이익누계액' : 231}
    if not is_std_account_exist(corp_code): # 표준계정 없으면, 자동 생성.
        original = 'template.xlsx'
        target = os.path.join("dart", "dart", corp_code, "financial_statements", "standard_account.xlsx")
        shutil.copyfile(original, target)
        print(corp_code, "표준계정 생성!")
        
    df = get_standard_account(corp_code, "재무상태표") # 해당 회사 표준계정 불러오기.
    files = get_db_files(corp_code, "재무상태표") # 해당 회사 DB의 전체 파일들 불러오기
    quarters = get_corp_db_quarters(corp_code, "재무상태표") # 파일들의 해당 분기별 리스트 데이터
    
    for i in range(len(files)):
        corp_income_dict = create_corp_income_dict(corp_code, files[i], "재무상태표") # 각 분기별 파일들의 딕셔너리 데이터.
        entry_iloc = get_entry_iloc(corp_code, files[i], "재무상태표") # 각 파일 항목들의 인덱스.

        for k in corp_income_dict: # 표준계정에 해당 데이터 입력
            if k == '자산총계':
                df.loc[std_account_entry_iloc['자산총계'],quarters[i]] = corp_income_dict['자산총계']
                print(corp_code, "자산총계", quarters[i], "표준계정에 업데이트 완료!")
            if k == '유동자산':
                df.loc[std_account_entry_iloc['유동자산'],quarters[i]] = corp_income_dict['유동자산']
                print(corp_code, "유동자산", quarters[i], "표준계정에 업데이트 완료!")
            elif k == '재고자산':
                df.loc[std_account_entry_iloc['재고자산'],quarters[i]] = corp_income_dict['재고자산']
                print(corp_code, "재고자산", quarters[i], "표준계정에 업데이트 완료!")
            elif k == '기타유동자산':
                df.loc[std_account_entry_iloc['기타유동자산'],quarters[i]] = corp_income_dict['기타유동자산']
                print(corp_code, "기타유동자산", quarters[i], "표준계정에 업데이트 완료!")
            elif k == '유형자산':
                df.loc[std_account_entry_iloc['유형자산'],quarters[i]] = corp_income_dict['유형자산']
                print(corp_code, "유형자산", quarters[i], "표준계정에 업데이트 완료!")
            elif k == '무형자산':
                df.loc[std_account_entry_iloc['무형자산'],quarters[i]] = corp_income_dict['무형자산']
                print(corp_code, "무형자산", quarters[i], "표준계정에 업데이트 완료!")
            elif k == '투자부동산':
                df.loc[std_account_entry_iloc['투자부동산'],quarters[i]] = corp_income_dict['투자부동산']
                print(corp_code, "투자부동산", quarters[i], "표준계정에 업데이트 완료!")
            elif k == '기타유동부채':
                df.loc[std_account_entry_iloc['기타유동부채'],quarters[i]] = corp_income_dict['기타유동부채']
                print(corp_code, "기타유동부채", quarters[i], "표준계정에 업데이트 완료!")
                
             #일치하지 않는 항목 업데이트 코드 => But, 시간 너무 오래 잡아먹음.
            else:
                with open('balance_sheet_dummy.txt', 'a', encoding="euc-kr") as f:
                    f.write(k)
                    f.write('\n')
                print("일치하지 않는 재무상태표 항목이름", k, "가 더미파일에 추가되엇습니다.")
            
                
                
        path = os.path.join("dart", "dart", corp_code, "financial_statements", "standard_account.xlsx")
    
        df1 = get_standard_account(corp_code, "포괄손익계산서") 
        df2 = get_standard_account(corp_code, "현금흐름표")
        with pd.ExcelWriter(path) as writer:
            df.to_excel(writer, encoding="euc-kr", sheet_name = "재무상태표", index = False) # 업데이트한 데이터를 저장
            df1.to_excel(writer, sheet_name="포괄손익계산서", index = False) # 건드리지 않고 바로 시트에 저장.
            df2.to_excel(writer, sheet_name="현금흐름표", index = False) # 건드리지 않고 바로 시트에 저장.
#update_balance_sheet("000020")

000020 유동자산 2017.q2(항목) 표준계정에 업데이트 완료!
일치하지 않는 재무상태표 항목이름 현금및현금성자산 가 더미파일에 추가되엇습니다.
일치하지 않는 재무상태표 항목이름 기타금융자산 가 더미파일에 추가되엇습니다.
일치하지 않는 재무상태표 항목이름 매출채권 및 기타채권 가 더미파일에 추가되엇습니다.
000020 재고자산 2017.q2(항목) 표준계정에 업데이트 완료!
000020 기타유동자산 2017.q2(항목) 표준계정에 업데이트 완료!
일치하지 않는 재무상태표 항목이름 비유동자산 가 더미파일에 추가되엇습니다.
일치하지 않는 재무상태표 항목이름 매도가능금융자산 가 더미파일에 추가되엇습니다.
일치하지 않는 재무상태표 항목이름 관계기업투자주식 가 더미파일에 추가되엇습니다.
000020 유형자산 2017.q2(항목) 표준계정에 업데이트 완료!
000020 무형자산 2017.q2(항목) 표준계정에 업데이트 완료!
일치하지 않는 재무상태표 항목이름 이연법인세자산 가 더미파일에 추가되엇습니다.
000020 자산총계 2017.q2(항목) 표준계정에 업데이트 완료!
일치하지 않는 재무상태표 항목이름 자산총계 가 더미파일에 추가되엇습니다.
일치하지 않는 재무상태표 항목이름 유동부채 가 더미파일에 추가되엇습니다.
일치하지 않는 재무상태표 항목이름 매입채무 및 기타채무 가 더미파일에 추가되엇습니다.
일치하지 않는 재무상태표 항목이름 미지급 법인세 가 더미파일에 추가되엇습니다.
000020 기타유동부채 2017.q2(항목) 표준계정에 업데이트 완료!
일치하지 않는 재무상태표 항목이름 비유동부채 가 더미파일에 추가되엇습니다.
일치하지 않는 재무상태표 항목이름 기타금융부채 가 더미파일에 추가되엇습니다.
일치하지 않는 재무상태표 항목이름 확정급여부채 가 더미파일에 추가되엇습니다.
일치하지 않는 재무상태표 항목이름 부채총계 가 더미파일에 추가되엇습니다.
일치하지 않는 재무상태표 항목이름 납입자본 가 더미파일에 추가되엇습니다.
일치하지 않는 재무상태표 항목이름 자본금 가 더미

000020 유동자산 2018.q4(항목) 표준계정에 업데이트 완료!
일치하지 않는 재무상태표 항목이름 현금및현금성자산 가 더미파일에 추가되엇습니다.
일치하지 않는 재무상태표 항목이름 기타유동금융자산 가 더미파일에 추가되엇습니다.
일치하지 않는 재무상태표 항목이름 매출채권 및 기타채권 가 더미파일에 추가되엇습니다.
000020 재고자산 2018.q4(항목) 표준계정에 업데이트 완료!
000020 기타유동자산 2018.q4(항목) 표준계정에 업데이트 완료!
일치하지 않는 재무상태표 항목이름 비유동자산 가 더미파일에 추가되엇습니다.
일치하지 않는 재무상태표 항목이름 매도가능금융자산 가 더미파일에 추가되엇습니다.
일치하지 않는 재무상태표 항목이름 지분법적용 투자주식 가 더미파일에 추가되엇습니다.
일치하지 않는 재무상태표 항목이름 기타비유동금융자산 가 더미파일에 추가되엇습니다.
000020 유형자산 2018.q4(항목) 표준계정에 업데이트 완료!
000020 무형자산 2018.q4(항목) 표준계정에 업데이트 완료!
일치하지 않는 재무상태표 항목이름 이연법인세자산 가 더미파일에 추가되엇습니다.
000020 자산총계 2018.q4(항목) 표준계정에 업데이트 완료!
일치하지 않는 재무상태표 항목이름 자산총계 가 더미파일에 추가되엇습니다.
일치하지 않는 재무상태표 항목이름 유동부채 가 더미파일에 추가되엇습니다.
일치하지 않는 재무상태표 항목이름 매입채무 및 기타유동채무 가 더미파일에 추가되엇습니다.
일치하지 않는 재무상태표 항목이름 미지급법인세 가 더미파일에 추가되엇습니다.
000020 기타유동부채 2018.q4(항목) 표준계정에 업데이트 완료!
일치하지 않는 재무상태표 항목이름 비유동부채 가 더미파일에 추가되엇습니다.
일치하지 않는 재무상태표 항목이름 장기매입채무 및 기타비유동채무 가 더미파일에 추가되엇습니다.
일치하지 않는 재무상태표 항목이름 기타비유동금융부채 가 더미파일에 추가되엇습니다.
일치하지 않는 재무상태표 항목이름 확정급여채무 가 더미파일에 추가되엇습니다.
일치

000020 유동자산 2020.q1(항목) 표준계정에 업데이트 완료!
일치하지 않는 재무상태표 항목이름 현금및현금성자산 가 더미파일에 추가되엇습니다.
일치하지 않는 재무상태표 항목이름 기타유동금융자산 가 더미파일에 추가되엇습니다.
일치하지 않는 재무상태표 항목이름 매출채권 및 기타채권 가 더미파일에 추가되엇습니다.
000020 재고자산 2020.q1(항목) 표준계정에 업데이트 완료!
000020 기타유동자산 2020.q1(항목) 표준계정에 업데이트 완료!
일치하지 않는 재무상태표 항목이름 당기법인세자산 가 더미파일에 추가되엇습니다.
일치하지 않는 재무상태표 항목이름 비유동자산 가 더미파일에 추가되엇습니다.
일치하지 않는 재무상태표 항목이름 매도가능금융자산 가 더미파일에 추가되엇습니다.
일치하지 않는 재무상태표 항목이름 기타비유동금융자산 가 더미파일에 추가되엇습니다.
000020 유형자산 2020.q1(항목) 표준계정에 업데이트 완료!
000020 무형자산 2020.q1(항목) 표준계정에 업데이트 완료!
일치하지 않는 재무상태표 항목이름 사용권자산 가 더미파일에 추가되엇습니다.
일치하지 않는 재무상태표 항목이름 이연법인세자산 가 더미파일에 추가되엇습니다.
000020 자산총계 2020.q1(항목) 표준계정에 업데이트 완료!
일치하지 않는 재무상태표 항목이름 자산총계 가 더미파일에 추가되엇습니다.
일치하지 않는 재무상태표 항목이름 유동부채 가 더미파일에 추가되엇습니다.
일치하지 않는 재무상태표 항목이름 매입채무 및 기타유동채무 가 더미파일에 추가되엇습니다.
일치하지 않는 재무상태표 항목이름 미지급법인세 가 더미파일에 추가되엇습니다.
000020 기타유동부채 2020.q1(항목) 표준계정에 업데이트 완료!
일치하지 않는 재무상태표 항목이름 비유동부채 가 더미파일에 추가되엇습니다.
일치하지 않는 재무상태표 항목이름 장기매입채무 및 기타비유동채무 가 더미파일에 추가되엇습니다.
일치하지 않는 재무상태표 항목이름 기타비유동금융부채 가 더미파일에 추가되엇습니다.
일치하지 않

In [12]:
def update_cash_flow(corp_code):
    # 표준계정 항목들의 인덱스 모음.
    std_account_entry_iloc = {'영업활동으로인한현금흐름' : 0, '당기순이익': 1, '법인세비용차감전계속사업이익' : 2, 
                              '현금유출이없는비용등가산' : 3,'현금유이입이없는수익등차감' : 46, 
                              '영업활동으로인한자산부채변동(운전자본변동)' : 85, '투자활동현금유입액' : 144,
                             '투자활동현금유출액' : 184, '재무활동현금유입액' : 242, '재무활동현금유출액' : 271}
    if not is_std_account_exist(corp_code): # 표준계정 없으면, 자동 생성.
        original = 'template.xlsx'
        target = os.path.join("dart", "dart", corp_code, "financial_statements", "standard_account.xlsx")
        shutil.copyfile(original, target)
        print(corp_code, "표준계정 생성!")
        
    df = get_standard_account(corp_code, "현금흐름표") # 해당 회사 표준계정 불러오기.
    files = get_db_files(corp_code, "현금흐름표") # 해당 회사 DB의 전체 파일들 불러오기
    quarters = get_corp_db_quarters(corp_code, "현금흐름표") # 파일들의 해당 분기별 리스트 데이터
    
    for i in range(len(files)):
        corp_income_dict = create_corp_income_dict(corp_code, files[i], "현금흐름표") # 각 분기별 파일들의 딕셔너리 데이터.
        entry_iloc = get_entry_iloc(corp_code, files[i], "현금흐름표") # 각 파일 항목들의 인덱스.

        for k in corp_income_dict: # 표준계정에 해당 데이터 입력
            if k == '영업활동현금흐름':
                df.loc[std_account_entry_iloc['영업활동으로인한현금흐름'],quarters[i]] = corp_income_dict['영업활동현금흐름']
                print(corp_code, "영업활동으로인한현금흐름", quarters[i], "표준계정에 업데이트 완료!")
            elif k == '영업활동으로 인한 현금흐름':
                df.loc[std_account_entry_iloc['영업활동으로인한현금흐름'],quarters[i]] = corp_income_dict['영업활동으로 인한 현금흐름']
                print(corp_code, "영업활동으로인한현금흐름", quarters[i], "표준계정에 업데이트 완료!")
            elif k == '당기순이익':
                df.loc[std_account_entry_iloc['당기순이익'],quarters[i]] = corp_income_dict['당기순이익']
                print(corp_code, "당기순이익", quarters[i], "표준계정에 업데이트 완료!")
            elif k == '당기순이익(손실)':
                df.loc[std_account_entry_iloc['당기순이익'],quarters[i]] = corp_income_dict['당기순이익(손실)']
                print(corp_code, "당기순이익", quarters[i], "표준계정에 업데이트 완료!")
            elif k == '분기순이익':
                df.loc[std_account_entry_iloc['당기순이익'],quarters[i]] = corp_income_dict['분기순이익']
                print(corp_code, "당기순이익", quarters[i], "표준계정에 업데이트 완료!")
            elif k == '분기순이익(손실)':
                df.loc[std_account_entry_iloc['당기순이익'],quarters[i]] = corp_income_dict['분기순이익(손실)']
                print(corp_code, "당기순이익", quarters[i], "표준계정에 업데이트 완료!")
                
            
            # 일치하지 않는 항목 업데이트 코드 => But, 시간 너무 오래 잡아먹음.
            else:
                with open('cash_flow_dummy.txt', 'a', encoding="euc-kr") as f:
                    f.write(k)
                    f.write('\n')
                print("일치하지 않는 재무상태표 항목이름", k, "가 더미파일에 추가되었습니다.")
               
        path = os.path.join("dart", "dart", corp_code, "financial_statements", "standard_account.xlsx")
    
        df1 = get_standard_account(corp_code, "포괄손익계산서")
        df2 = get_standard_account(corp_code, "재무상태표")
        with pd.ExcelWriter(path) as writer:
            df.to_excel(writer, encoding="euc-kr", sheet_name = "현금흐름표", index = False) # 저장
            df1.to_excel(writer, sheet_name="포괄손익계산서", index = False)
            df2.to_excel(writer, sheet_name="재무상태표", index = False)
#update_cash_flow("000140")

000140 영업활동으로인한현금흐름 2021.q2(항목) 표준계정에 업데이트 완료!
000140 당기순이익 2021.q2(항목) 표준계정에 업데이트 완료!
일치하지 않는 재무상태표 항목이름 당기순이익조정을 위한 가감 가 더미파일에 추가되었습니다.
일치하지 않는 재무상태표 항목이름 외화환산이익 가 더미파일에 추가되었습니다.
일치하지 않는 재무상태표 항목이름 외화환산손실 가 더미파일에 추가되었습니다.
일치하지 않는 재무상태표 항목이름 대손상각비 가 더미파일에 추가되었습니다.
일치하지 않는 재무상태표 항목이름 재고자산감액 가 더미파일에 추가되었습니다.
일치하지 않는 재무상태표 항목이름 감가상각비(투자부동산) 가 더미파일에 추가되었습니다.
일치하지 않는 재무상태표 항목이름 감가상각비 가 더미파일에 추가되었습니다.
일치하지 않는 재무상태표 항목이름 유형자산처분이익 가 더미파일에 추가되었습니다.
일치하지 않는 재무상태표 항목이름 유형자산처분손실 가 더미파일에 추가되었습니다.
일치하지 않는 재무상태표 항목이름 투자부동산처분이익 가 더미파일에 추가되었습니다.
일치하지 않는 재무상태표 항목이름 투자부동산처분손실 가 더미파일에 추가되었습니다.
일치하지 않는 재무상태표 항목이름 무형자산처분이익 가 더미파일에 추가되었습니다.
일치하지 않는 재무상태표 항목이름 무형자산처분손실 가 더미파일에 추가되었습니다.
일치하지 않는 재무상태표 항목이름 무형자산상각비 가 더미파일에 추가되었습니다.
일치하지 않는 재무상태표 항목이름 퇴직급여 가 더미파일에 추가되었습니다.
일치하지 않는 재무상태표 항목이름 파생상품평가손실 가 더미파일에 추가되었습니다.
일치하지 않는 재무상태표 항목이름 파생상품거래이익 가 더미파일에 추가되었습니다.
일치하지 않는 재무상태표 항목이름 파생상품거래손실 가 더미파일에 추가되었습니다.
일치하지 않는 재무상태표 항목이름 이자수익 가 더미파일에 추가되었습니다.
일치하지 않는 재무상태표 항목이름 배당금수익 가 더미파일에 추가되었습니다.
일치하지 않는 재무상태표 항목이름 이자비용 가

000140 영업활동으로인한현금흐름 2020.q4(항목) 표준계정에 업데이트 완료!
000140 당기순이익 2020.q4(항목) 표준계정에 업데이트 완료!
일치하지 않는 재무상태표 항목이름 당기순이익조정을 위한 가감 가 더미파일에 추가되었습니다.
일치하지 않는 재무상태표 항목이름 외화환산이익 가 더미파일에 추가되었습니다.
일치하지 않는 재무상태표 항목이름 외화환산손실 가 더미파일에 추가되었습니다.
일치하지 않는 재무상태표 항목이름 대손상각비 가 더미파일에 추가되었습니다.
일치하지 않는 재무상태표 항목이름 재고자산폐기손실 가 더미파일에 추가되었습니다.
일치하지 않는 재무상태표 항목이름 매각예정비유동자산손상차손 가 더미파일에 추가되었습니다.
일치하지 않는 재무상태표 항목이름 매각예정비유동자산처분손실 가 더미파일에 추가되었습니다.
일치하지 않는 재무상태표 항목이름 매도가능금융자산처분이익 가 더미파일에 추가되었습니다.
일치하지 않는 재무상태표 항목이름 매도가능금융자산손상차손 가 더미파일에 추가되었습니다.
일치하지 않는 재무상태표 항목이름 매도가능금융자산처분손실 가 더미파일에 추가되었습니다.
일치하지 않는 재무상태표 항목이름 종속기업투자처분이익 가 더미파일에 추가되었습니다.
일치하지 않는 재무상태표 항목이름 관계기업지분법이익 가 더미파일에 추가되었습니다.
일치하지 않는 재무상태표 항목이름 관계기업투자처분이익 가 더미파일에 추가되었습니다.
일치하지 않는 재무상태표 항목이름 감가상각비(투자부동산) 가 더미파일에 추가되었습니다.
일치하지 않는 재무상태표 항목이름 투자부동산처분이익 가 더미파일에 추가되었습니다.
일치하지 않는 재무상태표 항목이름 투자부동산처분손실 가 더미파일에 추가되었습니다.
일치하지 않는 재무상태표 항목이름 감가상각비 가 더미파일에 추가되었습니다.
일치하지 않는 재무상태표 항목이름 유형자산처분이익 가 더미파일에 추가되었습니다.
일치하지 않는 재무상태표 항목이름 유형자산처분손실 가 더미파일에 추가되었습니다.
일치하지 않는 재무상태표 항목이름 무형자산상각비 가 

000140 영업활동으로인한현금흐름 2021.q3(항목) 표준계정에 업데이트 완료!
000140 당기순이익 2021.q3(항목) 표준계정에 업데이트 완료!
일치하지 않는 재무상태표 항목이름 당기순이익조정을 위한 가감 가 더미파일에 추가되었습니다.
일치하지 않는 재무상태표 항목이름 외화환산이익 가 더미파일에 추가되었습니다.
일치하지 않는 재무상태표 항목이름 외화환산손실 가 더미파일에 추가되었습니다.
일치하지 않는 재무상태표 항목이름 대손상각비 가 더미파일에 추가되었습니다.
일치하지 않는 재무상태표 항목이름 재고자산폐기손실 가 더미파일에 추가되었습니다.
일치하지 않는 재무상태표 항목이름 매도가능금융자산처분이익 가 더미파일에 추가되었습니다.
일치하지 않는 재무상태표 항목이름 매도가능금융자산처분손실 가 더미파일에 추가되었습니다.
일치하지 않는 재무상태표 항목이름 당기손익-공정가치측정금융자산평가이익 가 더미파일에 추가되었습니다.
일치하지 않는 재무상태표 항목이름 당기손익-공정가치측정금융자산평가손실 가 더미파일에 추가되었습니다.
일치하지 않는 재무상태표 항목이름 관계기업지분법이익 가 더미파일에 추가되었습니다.
일치하지 않는 재무상태표 항목이름 감가상각비(투자부동산) 가 더미파일에 추가되었습니다.
일치하지 않는 재무상태표 항목이름 감가상각비 가 더미파일에 추가되었습니다.
일치하지 않는 재무상태표 항목이름 유형자산처분이익 가 더미파일에 추가되었습니다.
일치하지 않는 재무상태표 항목이름 유형자산처분손실 가 더미파일에 추가되었습니다.
일치하지 않는 재무상태표 항목이름 무형자산상각비 가 더미파일에 추가되었습니다.
일치하지 않는 재무상태표 항목이름 무형자산처분이익 가 더미파일에 추가되었습니다.
일치하지 않는 재무상태표 항목이름 무형자산처분손실 가 더미파일에 추가되었습니다.
일치하지 않는 재무상태표 항목이름 퇴직급여 가 더미파일에 추가되었습니다.
일치하지 않는 재무상태표 항목이름 이자수익 가 더미파일에 추가되었습니다.
일치하지 않는 재무상태표 항목이름 배당금수익 가 더미파일에 추가되

000140 영업활동으로인한현금흐름 2021.q1(항목) 표준계정에 업데이트 완료!
000140 당기순이익 2021.q1(항목) 표준계정에 업데이트 완료!
일치하지 않는 재무상태표 항목이름 당기순이익조정을 위한 가감 가 더미파일에 추가되었습니다.
일치하지 않는 재무상태표 항목이름 외화환산이익 가 더미파일에 추가되었습니다.
일치하지 않는 재무상태표 항목이름 외화환산손실 가 더미파일에 추가되었습니다.
일치하지 않는 재무상태표 항목이름 대손상각비 가 더미파일에 추가되었습니다.
일치하지 않는 재무상태표 항목이름 재고자산폐기손실 가 더미파일에 추가되었습니다.
일치하지 않는 재무상태표 항목이름 당기손익-공정가치측정금융자산평가이익 가 더미파일에 추가되었습니다.
일치하지 않는 재무상태표 항목이름 당기손익-공정가치측정금융자산평가손실 가 더미파일에 추가되었습니다.
일치하지 않는 재무상태표 항목이름 관계기업지분법손실(이익) 가 더미파일에 추가되었습니다.
일치하지 않는 재무상태표 항목이름 감가상각비(투자부동산) 가 더미파일에 추가되었습니다.
일치하지 않는 재무상태표 항목이름 감가상각비 가 더미파일에 추가되었습니다.
일치하지 않는 재무상태표 항목이름 유형자산처분이익 가 더미파일에 추가되었습니다.
일치하지 않는 재무상태표 항목이름 유형자산처분손실 가 더미파일에 추가되었습니다.
일치하지 않는 재무상태표 항목이름 무형자산상각비 가 더미파일에 추가되었습니다.
일치하지 않는 재무상태표 항목이름 무형자산처분이익 가 더미파일에 추가되었습니다.
일치하지 않는 재무상태표 항목이름 무형자산처분손실 가 더미파일에 추가되었습니다.
일치하지 않는 재무상태표 항목이름 감가상각비(사용권자산) 가 더미파일에 추가되었습니다.
일치하지 않는 재무상태표 항목이름 기타의 대손상각비 가 더미파일에 추가되었습니다.
일치하지 않는 재무상태표 항목이름 퇴직급여 가 더미파일에 추가되었습니다.
일치하지 않는 재무상태표 항목이름 이자수익 가 더미파일에 추가되었습니다.
일치하지 않는 재무상태표 항목이름 배당금수익 가 더미파일에 추가

000140 영업활동으로인한현금흐름 2020.q2(항목) 표준계정에 업데이트 완료!
000140 당기순이익 2020.q2(항목) 표준계정에 업데이트 완료!
일치하지 않는 재무상태표 항목이름 당기순이익조정을 위한 가감 가 더미파일에 추가되었습니다.
일치하지 않는 재무상태표 항목이름 외화환산이익 가 더미파일에 추가되었습니다.
일치하지 않는 재무상태표 항목이름 외화환산손실 가 더미파일에 추가되었습니다.
일치하지 않는 재무상태표 항목이름 대손상각비 가 더미파일에 추가되었습니다.
일치하지 않는 재무상태표 항목이름 기타의 대손상각비 가 더미파일에 추가되었습니다.
일치하지 않는 재무상태표 항목이름 재고자산폐기손실 가 더미파일에 추가되었습니다.
일치하지 않는 재무상태표 항목이름 재고자산평가손실 가 더미파일에 추가되었습니다.
일치하지 않는 재무상태표 항목이름 당기손익-공정가치측정금융자산평가이익 가 더미파일에 추가되었습니다.
일치하지 않는 재무상태표 항목이름 당기손익-공정가치측정금융자산평가손실 가 더미파일에 추가되었습니다.
일치하지 않는 재무상태표 항목이름 지분법이익 가 더미파일에 추가되었습니다.
일치하지 않는 재무상태표 항목이름 감가상각비(투자부동산) 가 더미파일에 추가되었습니다.
일치하지 않는 재무상태표 항목이름 감가상각비 가 더미파일에 추가되었습니다.
일치하지 않는 재무상태표 항목이름 유형자산처분이익 가 더미파일에 추가되었습니다.
일치하지 않는 재무상태표 항목이름 유형자산처분손실 가 더미파일에 추가되었습니다.
일치하지 않는 재무상태표 항목이름 유형자산손상차손 가 더미파일에 추가되었습니다.
일치하지 않는 재무상태표 항목이름 무형자산상각비 가 더미파일에 추가되었습니다.
일치하지 않는 재무상태표 항목이름 무형자산처분이익 가 더미파일에 추가되었습니다.
일치하지 않는 재무상태표 항목이름 무형자산처분손실 가 더미파일에 추가되었습니다.
일치하지 않는 재무상태표 항목이름 감가상각비(사용권자산) 가 더미파일에 추가되었습니다.
일치하지 않는 재무상태표 항목이름 리스조정이익 가 더미파일에 추

000140 영업활동으로인한현금흐름 2020.q4(항목) 표준계정에 업데이트 완료!
000140 당기순이익 2020.q4(항목) 표준계정에 업데이트 완료!
일치하지 않는 재무상태표 항목이름 당기순이익조정을 위한 가감 가 더미파일에 추가되었습니다.
일치하지 않는 재무상태표 항목이름 외화환산이익 가 더미파일에 추가되었습니다.
일치하지 않는 재무상태표 항목이름 외화환산손실 가 더미파일에 추가되었습니다.
일치하지 않는 재무상태표 항목이름 대손상각비 가 더미파일에 추가되었습니다.
일치하지 않는 재무상태표 항목이름 기타의 대손상각비 가 더미파일에 추가되었습니다.
일치하지 않는 재무상태표 항목이름 재고자산폐기손실 가 더미파일에 추가되었습니다.
일치하지 않는 재무상태표 항목이름 재고자산평가손실 가 더미파일에 추가되었습니다.
일치하지 않는 재무상태표 항목이름 당기손익-공정가치측정금융자산평가이익 가 더미파일에 추가되었습니다.
일치하지 않는 재무상태표 항목이름 당기손익-공정가치측정금융자산평가손실 가 더미파일에 추가되었습니다.
일치하지 않는 재무상태표 항목이름 지분법이익 가 더미파일에 추가되었습니다.
일치하지 않는 재무상태표 항목이름 감가상각비(투자부동산) 가 더미파일에 추가되었습니다.
일치하지 않는 재무상태표 항목이름 감가상각비 가 더미파일에 추가되었습니다.
일치하지 않는 재무상태표 항목이름 유형자산처분이익 가 더미파일에 추가되었습니다.
일치하지 않는 재무상태표 항목이름 유형자산처분손실 가 더미파일에 추가되었습니다.
일치하지 않는 재무상태표 항목이름 유형자산손상차손 가 더미파일에 추가되었습니다.
일치하지 않는 재무상태표 항목이름 무형자산상각비 가 더미파일에 추가되었습니다.
일치하지 않는 재무상태표 항목이름 무형자산처분이익 가 더미파일에 추가되었습니다.
일치하지 않는 재무상태표 항목이름 무형자산처분손실 가 더미파일에 추가되었습니다.
일치하지 않는 재무상태표 항목이름 무형자산손상차손 가 더미파일에 추가되었습니다.
일치하지 않는 재무상태표 항목이름 감가상각비(사용권자산) 가 더미파일에

In [10]:
def convert_datetime(date):
    l = date.split(" ")
    year = l[-1]
    month = l[1]
    
    if month == "Jan":
        month = "01"
    elif month == "Feb":
        month = "02"
    elif month == "Mar":
        month = "03"
    elif month == "Apr":
        month = "04"
    elif month == "May":
        month = "05"
    elif month == "Jun":
        month = "06"
    elif month == "Jul":
        month = "07"
    elif month == "Aug":
        month = "08"
    elif month == "Sep":
        month = "09"
    elif month == "Oct":
        month = "10"
    elif month == "Nov":
        month = "11"
    elif month == "Dec":
        month = "12"
    day = l[2]
    
    return year + "-" + month + "-" + day

convert_datetime("Tue May 24 16:02:00 2021")

'2021-05-24'

In [11]:
def time_comparison(last_modified, date_today):
    if last_modified != date_today:
        return True
    else:
        return False

In [13]:
def db_corp_income_update():
    path = os.path.join("dart", "dart")
    companies = os.listdir(path)
    starting_time = datetime.datetime.now()
    for corp_code in companies:
        # last_modified ~ print("----")는 일단 전체 DB 업데이트 한번 다 한 이후에 실행할 것!
        
        # 해당 폴더 마지막으로 업데이트 된 시각
        last_modified = convert_datetime(time.ctime(os.path.getmtime(os.path.join("dart", "dart", corp_code, "financial_statements", "income_statement"))))
        # 현재 시각
        date_today = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S').split(" ")[0]
    
        if time_comparison(last_modified, date_today): # 오늘 추가된 새로운 파일이 없을 시에는 표준계정에 업데이트 안하고 넘어감.
            print(corp_code, "오늘 업데이트된 보고서가 없습니다.")
            continue
        else:
            print(corp_code, datetime.datetime.now(), "에 표준계정 업데이트 시작..." )
            update_cash_flow(corp_code)
            print(corp_code, datetime.datetime.now(), "에 표준계정 업데이트 완료!")
            print("-------------------------------------------------------------------------------------")
            
    end_time = datetime.datetime.now()
    print("전체 포괄손익계산서 DB화 작업 완료!")
    print("시작 시간: ", starting_time, "종료 시간: ", end_time)
db_corp_income_update()

000020 오늘 업데이트된 보고서가 없습니다.
000040 오늘 업데이트된 보고서가 없습니다.
000050 오늘 업데이트된 보고서가 없습니다.
000070 오늘 업데이트된 보고서가 없습니다.
000080 오늘 업데이트된 보고서가 없습니다.
000100 오늘 업데이트된 보고서가 없습니다.
000120 2021-08-27 16:50:37.410103 에 표준계정 업데이트 시작...
000120 영업활동으로인한현금흐름 2017.q2(항목) 표준계정에 업데이트 완료!
일치하지 않는 재무상태표 항목이름 계속영업이익 가 더미파일에 추가되었습니다.
일치하지 않는 재무상태표 항목이름 조정 가 더미파일에 추가되었습니다.
일치하지 않는 재무상태표 항목이름 법인세비용 가 더미파일에 추가되었습니다.
일치하지 않는 재무상태표 항목이름 감가상각비 가 더미파일에 추가되었습니다.
일치하지 않는 재무상태표 항목이름 퇴직급여 가 더미파일에 추가되었습니다.
일치하지 않는 재무상태표 항목이름 임차료 가 더미파일에 추가되었습니다.
일치하지 않는 재무상태표 항목이름 무형자산상각비 가 더미파일에 추가되었습니다.
일치하지 않는 재무상태표 항목이름 대손상각비 가 더미파일에 추가되었습니다.
일치하지 않는 재무상태표 항목이름 기타수취채권손상차손 가 더미파일에 추가되었습니다.
일치하지 않는 재무상태표 항목이름 이자수익 가 더미파일에 추가되었습니다.
일치하지 않는 재무상태표 항목이름 이자비용 가 더미파일에 추가되었습니다.
일치하지 않는 재무상태표 항목이름 배당금수익 가 더미파일에 추가되었습니다.
일치하지 않는 재무상태표 항목이름 외화환산이익 가 더미파일에 추가되었습니다.
일치하지 않는 재무상태표 항목이름 외화환산손실 가 더미파일에 추가되었습니다.
일치하지 않는 재무상태표 항목이름 금융보증비용 가 더미파일에 추가되었습니다.
일치하지 않는 재무상태표 항목이름 매도가능금융자산처분이익 가 더미파일에 추가되었습니다.
일치하지 않는 재무상태표 항목이름 매도가능금융자산처분손실 가 더미파일에 추가되었습니다.
일치하지 않는 

000120 영업활동으로인한현금흐름 2017.q3(항목) 표준계정에 업데이트 완료!
000120 당기순이익 2017.q3(항목) 표준계정에 업데이트 완료!
일치하지 않는 재무상태표 항목이름 비현금항목의 조정 가 더미파일에 추가되었습니다.
일치하지 않는 재무상태표 항목이름 법인세비용 가 더미파일에 추가되었습니다.
일치하지 않는 재무상태표 항목이름 감가상각비 가 더미파일에 추가되었습니다.
일치하지 않는 재무상태표 항목이름 퇴직급여 가 더미파일에 추가되었습니다.
일치하지 않는 재무상태표 항목이름 임차료 가 더미파일에 추가되었습니다.
일치하지 않는 재무상태표 항목이름 무형자산상각비 가 더미파일에 추가되었습니다.
일치하지 않는 재무상태표 항목이름 대손상각비 가 더미파일에 추가되었습니다.
일치하지 않는 재무상태표 항목이름 기타의 대손상각비 가 더미파일에 추가되었습니다.
일치하지 않는 재무상태표 항목이름 이자비용 가 더미파일에 추가되었습니다.
일치하지 않는 재무상태표 항목이름 외화환산손실 가 더미파일에 추가되었습니다.
일치하지 않는 재무상태표 항목이름 금융보증비용 가 더미파일에 추가되었습니다.
일치하지 않는 재무상태표 항목이름 매도가능금융자산처분손실 가 더미파일에 추가되었습니다.
일치하지 않는 재무상태표 항목이름 매도가능금융자산손상차손 가 더미파일에 추가되었습니다.
일치하지 않는 재무상태표 항목이름 관계기업투자투식처분손실 가 더미파일에 추가되었습니다.
일치하지 않는 재무상태표 항목이름 관계기업투자주식손상차손 가 더미파일에 추가되었습니다.
일치하지 않는 재무상태표 항목이름 종송기업투자주식처분이익 가 더미파일에 추가되었습니다.
일치하지 않는 재무상태표 항목이름 종속기업투자주식처분손실 가 더미파일에 추가되었습니다.
일치하지 않는 재무상태표 항목이름 유형자산처분손실 가 더미파일에 추가되었습니다.
일치하지 않는 재무상태표 항목이름 유형자산손상차손 가 더미파일에 추가되었습니다.
일치하지 않는 재무상태표 항목이름 무형자산처분손실 가 더미파일에 추가되었습니다.
일치하지 않는 재무상태표 

000120 영업활동으로인한현금흐름 2018.q1(항목) 표준계정에 업데이트 완료!
000120 당기순이익 2018.q1(항목) 표준계정에 업데이트 완료!
일치하지 않는 재무상태표 항목이름 비현금항목의 조정 가 더미파일에 추가되었습니다.
일치하지 않는 재무상태표 항목이름 법인세비용 가 더미파일에 추가되었습니다.
일치하지 않는 재무상태표 항목이름 감가상각비 가 더미파일에 추가되었습니다.
일치하지 않는 재무상태표 항목이름 퇴직급여 가 더미파일에 추가되었습니다.
일치하지 않는 재무상태표 항목이름 임차료 가 더미파일에 추가되었습니다.
일치하지 않는 재무상태표 항목이름 무형자산상각비 가 더미파일에 추가되었습니다.
일치하지 않는 재무상태표 항목이름 대손상각비 가 더미파일에 추가되었습니다.
일치하지 않는 재무상태표 항목이름 기타수취채권손상차손 가 더미파일에 추가되었습니다.
일치하지 않는 재무상태표 항목이름 이자수익 가 더미파일에 추가되었습니다.
일치하지 않는 재무상태표 항목이름 이자비용 가 더미파일에 추가되었습니다.
일치하지 않는 재무상태표 항목이름 배당금수익 가 더미파일에 추가되었습니다.
일치하지 않는 재무상태표 항목이름 외화환산이익 가 더미파일에 추가되었습니다.
일치하지 않는 재무상태표 항목이름 외화환산손실 가 더미파일에 추가되었습니다.
일치하지 않는 재무상태표 항목이름 금융보증비용 가 더미파일에 추가되었습니다.
일치하지 않는 재무상태표 항목이름 지분상품및채무상품처분손실 가 더미파일에 추가되었습니다.
일치하지 않는 재무상태표 항목이름 관계기업투자주식손상차손 가 더미파일에 추가되었습니다.
일치하지 않는 재무상태표 항목이름 종속기업투자주식처분손실 가 더미파일에 추가되었습니다.
일치하지 않는 재무상태표 항목이름 유형자산처분이익 가 더미파일에 추가되었습니다.
일치하지 않는 재무상태표 항목이름 유형자산처분손실 가 더미파일에 추가되었습니다.
일치하지 않는 재무상태표 항목이름 무형자산손상차손 가 더미파일에 추가되었습니다.
일치하지 않는 재무상태표 항목이름 매각예정자산처분손실 가 더

일치하지 않는 재무상태표 항목이름 비지배지분의 순증가에 따른 현금유입 가 더미파일에 추가되었습니다.
일치하지 않는 재무상태표 항목이름 현금및현금성자산의 증가(감소) 가 더미파일에 추가되었습니다.
일치하지 않는 재무상태표 항목이름 외화환산으로 인한 현금증감 가 더미파일에 추가되었습니다.
일치하지 않는 재무상태표 항목이름 기초의 현금및현금성자산 가 더미파일에 추가되었습니다.
일치하지 않는 재무상태표 항목이름 기말의 현금및현금성자산 가 더미파일에 추가되었습니다.
000120 영업활동으로인한현금흐름 2019.q2(항목) 표준계정에 업데이트 완료!
일치하지 않는 재무상태표 항목이름 반기순이익 가 더미파일에 추가되었습니다.
일치하지 않는 재무상태표 항목이름 비현금항목의 조정 가 더미파일에 추가되었습니다.
일치하지 않는 재무상태표 항목이름 운전자본조정 가 더미파일에 추가되었습니다.
일치하지 않는 재무상태표 항목이름 이자의 수취 가 더미파일에 추가되었습니다.
일치하지 않는 재무상태표 항목이름 배당금의 수취 가 더미파일에 추가되었습니다.
일치하지 않는 재무상태표 항목이름 이자의 지급 가 더미파일에 추가되었습니다.
일치하지 않는 재무상태표 항목이름 법인세의 납부 가 더미파일에 추가되었습니다.
일치하지 않는 재무상태표 항목이름 투자활동으로 인한 현금흐름 가 더미파일에 추가되었습니다.
일치하지 않는 재무상태표 항목이름 유형자산의 취득 가 더미파일에 추가되었습니다.
일치하지 않는 재무상태표 항목이름 유형자산의 처분 가 더미파일에 추가되었습니다.
일치하지 않는 재무상태표 항목이름 무형자산의 취득 가 더미파일에 추가되었습니다.
일치하지 않는 재무상태표 항목이름 무형자산의 처분 가 더미파일에 추가되었습니다.
일치하지 않는 재무상태표 항목이름 매각예정자산의 처분 가 더미파일에 추가되었습니다.
일치하지 않는 재무상태표 항목이름 기타금융자산의 취득 가 더미파일에 추가되었습니다.
일치하지 않는 재무상태표 항목이름 기타금융자산의 처분 가 더미파일에 추가되었습니다.
일치하지 않는 재무상태표 항목이름

000120 영업활동으로인한현금흐름 2020.q1(항목) 표준계정에 업데이트 완료!
000120 당기순이익 2020.q1(항목) 표준계정에 업데이트 완료!
일치하지 않는 재무상태표 항목이름 비현금항목의 조정 가 더미파일에 추가되었습니다.
일치하지 않는 재무상태표 항목이름 운전자본의 조정 가 더미파일에 추가되었습니다.
일치하지 않는 재무상태표 항목이름 이자의 수취 가 더미파일에 추가되었습니다.
일치하지 않는 재무상태표 항목이름 배당금의 수취 가 더미파일에 추가되었습니다.
일치하지 않는 재무상태표 항목이름 이자의 지급 가 더미파일에 추가되었습니다.
일치하지 않는 재무상태표 항목이름 법인세의 납부 가 더미파일에 추가되었습니다.
일치하지 않는 재무상태표 항목이름 투자활동으로 인한 현금흐름 가 더미파일에 추가되었습니다.
일치하지 않는 재무상태표 항목이름 유형자산의 취득 가 더미파일에 추가되었습니다.
일치하지 않는 재무상태표 항목이름 유형자산의 처분 가 더미파일에 추가되었습니다.
일치하지 않는 재무상태표 항목이름 무형자산의 취득 가 더미파일에 추가되었습니다.
일치하지 않는 재무상태표 항목이름 무형자산의 처분 가 더미파일에 추가되었습니다.
일치하지 않는 재무상태표 항목이름 매각예정자산의 처분 가 더미파일에 추가되었습니다.
일치하지 않는 재무상태표 항목이름 당기손익-공정가치 측정 금융자산의 취득 가 더미파일에 추가되었습니다.
일치하지 않는 재무상태표 항목이름 기타금융자산의 취득 가 더미파일에 추가되었습니다.
일치하지 않는 재무상태표 항목이름 기타금융자산의 처분 가 더미파일에 추가되었습니다.
일치하지 않는 재무상태표 항목이름 비유동기타금융자산의 취득 가 더미파일에 추가되었습니다.
일치하지 않는 재무상태표 항목이름 비유동기타금융자산의 처분 가 더미파일에 추가되었습니다.
일치하지 않는 재무상태표 항목이름 사업결합으로 인한 순현금흐름 가 더미파일에 추가되었습니다.
일치하지 않는 재무상태표 항목이름 재무활동으로 인한 현금흐름 가 더미파일에 추가되었습니다.
일치하지 않는 재무상태표 항

...

017960 오늘 업데이트된 보고서가 없습니다.
018000 오늘 업데이트된 보고서가 없습니다.
018120 오늘 업데이트된 보고서가 없습니다.
018250 오늘 업데이트된 보고서가 없습니다.
018260 오늘 업데이트된 보고서가 없습니다.
018290 오늘 업데이트된 보고서가 없습니다.
018310 오늘 업데이트된 보고서가 없습니다.
018470 오늘 업데이트된 보고서가 없습니다.
018500 오늘 업데이트된 보고서가 없습니다.
018620 오늘 업데이트된 보고서가 없습니다.
018670 오늘 업데이트된 보고서가 없습니다.
018680 오늘 업데이트된 보고서가 없습니다.
018700 오늘 업데이트된 보고서가 없습니다.
018880 오늘 업데이트된 보고서가 없습니다.
019010 오늘 업데이트된 보고서가 없습니다.
019170 오늘 업데이트된 보고서가 없습니다.
019180 오늘 업데이트된 보고서가 없습니다.
019210 오늘 업데이트된 보고서가 없습니다.
019440 오늘 업데이트된 보고서가 없습니다.
019490 오늘 업데이트된 보고서가 없습니다.
019540 오늘 업데이트된 보고서가 없습니다.
019570 오늘 업데이트된 보고서가 없습니다.
019660 오늘 업데이트된 보고서가 없습니다.
019680 오늘 업데이트된 보고서가 없습니다.
019770 오늘 업데이트된 보고서가 없습니다.
019990 오늘 업데이트된 보고서가 없습니다.
020000 오늘 업데이트된 보고서가 없습니다.
020120 오늘 업데이트된 보고서가 없습니다.
020150 오늘 업데이트된 보고서가 없습니다.
020180 오늘 업데이트된 보고서가 없습니다.
020400 오늘 업데이트된 보고서가 없습니다.
020560 오늘 업데이트된 보고서가 없습니다.
020710 오늘 업데이트된 보고서가 없습니다.
020760 오늘 업데이트된 보고서가 없습니다.
021040 오늘 업데이트된 보고서가 없습니다.
021050 오늘 업데이트된 보고서가 없습니다.
021080 오늘 업데이트된 보고서가 없습니다.
0

051910 오늘 업데이트된 보고서가 없습니다.
051980 오늘 업데이트된 보고서가 없습니다.
052020 오늘 업데이트된 보고서가 없습니다.
052190 오늘 업데이트된 보고서가 없습니다.
052220 오늘 업데이트된 보고서가 없습니다.
052260 오늘 업데이트된 보고서가 없습니다.
052300 오늘 업데이트된 보고서가 없습니다.
052330 오늘 업데이트된 보고서가 없습니다.
052400 오늘 업데이트된 보고서가 없습니다.
052420 오늘 업데이트된 보고서가 없습니다.
052460 오늘 업데이트된 보고서가 없습니다.
052600 오늘 업데이트된 보고서가 없습니다.
052670 오늘 업데이트된 보고서가 없습니다.
052690 오늘 업데이트된 보고서가 없습니다.
052710 오늘 업데이트된 보고서가 없습니다.
052770 오늘 업데이트된 보고서가 없습니다.
052790 오늘 업데이트된 보고서가 없습니다.
052860 오늘 업데이트된 보고서가 없습니다.
052900 오늘 업데이트된 보고서가 없습니다.
053030 오늘 업데이트된 보고서가 없습니다.
053050 오늘 업데이트된 보고서가 없습니다.
053060 오늘 업데이트된 보고서가 없습니다.
053080 오늘 업데이트된 보고서가 없습니다.
053110 오늘 업데이트된 보고서가 없습니다.
053160 오늘 업데이트된 보고서가 없습니다.
053210 오늘 업데이트된 보고서가 없습니다.
053260 오늘 업데이트된 보고서가 없습니다.
053270 오늘 업데이트된 보고서가 없습니다.
053280 오늘 업데이트된 보고서가 없습니다.
053290 오늘 업데이트된 보고서가 없습니다.
053300 오늘 업데이트된 보고서가 없습니다.
053350 오늘 업데이트된 보고서가 없습니다.
053450 오늘 업데이트된 보고서가 없습니다.
053580 오늘 업데이트된 보고서가 없습니다.
053590 오늘 업데이트된 보고서가 없습니다.
053610 오늘 업데이트된 보고서가 없습니다.
053620 오늘 업데이트된 보고서가 없습니다.
0

084870 오늘 업데이트된 보고서가 없습니다.
084990 오늘 업데이트된 보고서가 없습니다.
085310 오늘 업데이트된 보고서가 없습니다.
085370 오늘 업데이트된 보고서가 없습니다.
085660 오늘 업데이트된 보고서가 없습니다.
085670 오늘 업데이트된 보고서가 없습니다.
085810 오늘 업데이트된 보고서가 없습니다.
085910 오늘 업데이트된 보고서가 없습니다.
086040 오늘 업데이트된 보고서가 없습니다.
086060 오늘 업데이트된 보고서가 없습니다.
086250 오늘 업데이트된 보고서가 없습니다.
086280 오늘 업데이트된 보고서가 없습니다.
086390 오늘 업데이트된 보고서가 없습니다.
086450 오늘 업데이트된 보고서가 없습니다.
086520 오늘 업데이트된 보고서가 없습니다.
086670 오늘 업데이트된 보고서가 없습니다.
086710 오늘 업데이트된 보고서가 없습니다.
086820 오늘 업데이트된 보고서가 없습니다.
086890 오늘 업데이트된 보고서가 없습니다.
086900 오늘 업데이트된 보고서가 없습니다.
086960 오늘 업데이트된 보고서가 없습니다.
086980 오늘 업데이트된 보고서가 없습니다.
087010 오늘 업데이트된 보고서가 없습니다.
087260 오늘 업데이트된 보고서가 없습니다.
087600 오늘 업데이트된 보고서가 없습니다.
087730 오늘 업데이트된 보고서가 없습니다.
088130 오늘 업데이트된 보고서가 없습니다.
088260 오늘 업데이트된 보고서가 없습니다.
088290 오늘 업데이트된 보고서가 없습니다.
088390 오늘 업데이트된 보고서가 없습니다.
088790 오늘 업데이트된 보고서가 없습니다.
088800 오늘 업데이트된 보고서가 없습니다.
088910 오늘 업데이트된 보고서가 없습니다.
089010 오늘 업데이트된 보고서가 없습니다.
089030 오늘 업데이트된 보고서가 없습니다.
089140 오늘 업데이트된 보고서가 없습니다.
089150 오늘 업데이트된 보고서가 없습니다.
0

153360 오늘 업데이트된 보고서가 없습니다.
153460 오늘 업데이트된 보고서가 없습니다.
153490 오늘 업데이트된 보고서가 없습니다.
153710 오늘 업데이트된 보고서가 없습니다.
154030 오늘 업데이트된 보고서가 없습니다.
154040 오늘 업데이트된 보고서가 없습니다.
155650 오늘 업데이트된 보고서가 없습니다.
155660 오늘 업데이트된 보고서가 없습니다.
155900 오늘 업데이트된 보고서가 없습니다.
156100 오늘 업데이트된 보고서가 없습니다.
158310 오늘 업데이트된 보고서가 없습니다.
158430 오늘 업데이트된 보고서가 없습니다.
159580 오늘 업데이트된 보고서가 없습니다.
159910 오늘 업데이트된 보고서가 없습니다.
160550 오늘 업데이트된 보고서가 없습니다.
160600 오늘 업데이트된 보고서가 없습니다.
160980 오늘 업데이트된 보고서가 없습니다.
161000 오늘 업데이트된 보고서가 없습니다.
161390 오늘 업데이트된 보고서가 없습니다.
161570 오늘 업데이트된 보고서가 없습니다.
161580 오늘 업데이트된 보고서가 없습니다.
161890 오늘 업데이트된 보고서가 없습니다.
163560 오늘 업데이트된 보고서가 없습니다.
163730 오늘 업데이트된 보고서가 없습니다.
164060 오늘 업데이트된 보고서가 없습니다.
166090 오늘 업데이트된 보고서가 없습니다.
166480 오늘 업데이트된 보고서가 없습니다.
168330 오늘 업데이트된 보고서가 없습니다.
169330 오늘 업데이트된 보고서가 없습니다.
170030 오늘 업데이트된 보고서가 없습니다.
170790 오늘 업데이트된 보고서가 없습니다.
170900 오늘 업데이트된 보고서가 없습니다.
170920 오늘 업데이트된 보고서가 없습니다.
171010 오늘 업데이트된 보고서가 없습니다.
171090 오늘 업데이트된 보고서가 없습니다.
171120 오늘 업데이트된 보고서가 없습니다.
172580 오늘 업데이트된 보고서가 없습니다.
1

313760 오늘 업데이트된 보고서가 없습니다.
314130 오늘 업데이트된 보고서가 없습니다.
314930 오늘 업데이트된 보고서가 없습니다.
317120 오늘 업데이트된 보고서가 없습니다.
317240 오늘 업데이트된 보고서가 없습니다.
317330 오늘 업데이트된 보고서가 없습니다.
317400 오늘 업데이트된 보고서가 없습니다.
317530 오늘 업데이트된 보고서가 없습니다.
317690 오늘 업데이트된 보고서가 없습니다.
317770 오늘 업데이트된 보고서가 없습니다.
317830 오늘 업데이트된 보고서가 없습니다.
317850 오늘 업데이트된 보고서가 없습니다.
317870 오늘 업데이트된 보고서가 없습니다.
318000 오늘 업데이트된 보고서가 없습니다.
318010 오늘 업데이트된 보고서가 없습니다.
318020 오늘 업데이트된 보고서가 없습니다.
318410 오늘 업데이트된 보고서가 없습니다.
319400 오늘 업데이트된 보고서가 없습니다.
319660 오늘 업데이트된 보고서가 없습니다.
320000 오늘 업데이트된 보고서가 없습니다.
321550 오늘 업데이트된 보고서가 없습니다.
321820 오늘 업데이트된 보고서가 없습니다.
322000 오늘 업데이트된 보고서가 없습니다.
322180 오늘 업데이트된 보고서가 없습니다.
322310 오늘 업데이트된 보고서가 없습니다.
322510 오늘 업데이트된 보고서가 없습니다.
322780 오늘 업데이트된 보고서가 없습니다.
323230 오늘 업데이트된 보고서가 없습니다.
323990 오늘 업데이트된 보고서가 없습니다.
326030 오늘 업데이트된 보고서가 없습니다.
327260 오늘 업데이트된 보고서가 없습니다.
330350 오늘 업데이트된 보고서가 없습니다.
330590 오늘 업데이트된 보고서가 없습니다.
330860 오늘 업데이트된 보고서가 없습니다.
331920 오늘 업데이트된 보고서가 없습니다.
332370 오늘 업데이트된 보고서가 없습니다.
332570 오늘 업데이트된 보고서가 없습니다.
3

In [14]:
def db_balance_sheet_update():
    path = os.path.join("dart", "dart")
    companies = os.listdir(path)
    starting_time = datetime.datetime.now()
    for corp_code in companies:
        # last_modified ~ print("----")는 일단 전체 DB 업데이트 한번 다 한 이후에 실행할 것!
        
        # 해당 폴더 마지막으로 업데이트 된 시각
        last_modified = convert_datetime(time.ctime(os.path.getmtime(os.path.join("dart", "dart", corp_code, "financial_statements", "balance_sheet"))))
        # 현재 시각
        date_today = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S').split(" ")[0]
    
        if time_comparison(last_modified, date_today):
            print(corp_code, "오늘 업데이트된 보고서가 없습니다.")
            continue
        else:
            print(corp_code, datetime.datetime.now(), "에 표준계정 업데이트 시작..." )
            update_cash_flow(corp_code)
            print(corp_code, datetime.datetime.now(), "에 표준계정 업데이트 완료!")
            print("-------------------------------------------------------------------------------------")
            
    end_time = datetime.datetime.now()
    print("전체 재무상태표 DB화 작업 완료!")
    print("시작 시간: ", starting_time, "종료 시간: ", end_time)
db_balance_sheet_update()

000020 오늘 업데이트된 보고서가 없습니다.
000040 오늘 업데이트된 보고서가 없습니다.
000050 오늘 업데이트된 보고서가 없습니다.
000070 오늘 업데이트된 보고서가 없습니다.
000080 오늘 업데이트된 보고서가 없습니다.
000100 오늘 업데이트된 보고서가 없습니다.
000120 오늘 업데이트된 보고서가 없습니다.
000140 오늘 업데이트된 보고서가 없습니다.
000150 오늘 업데이트된 보고서가 없습니다.
000180 오늘 업데이트된 보고서가 없습니다.
000210 오늘 업데이트된 보고서가 없습니다.
000220 오늘 업데이트된 보고서가 없습니다.
000230 오늘 업데이트된 보고서가 없습니다.
000240 오늘 업데이트된 보고서가 없습니다.
000250 오늘 업데이트된 보고서가 없습니다.
000270 오늘 업데이트된 보고서가 없습니다.
000300 오늘 업데이트된 보고서가 없습니다.
000320 오늘 업데이트된 보고서가 없습니다.
000390 오늘 업데이트된 보고서가 없습니다.
000430 오늘 업데이트된 보고서가 없습니다.
000440 오늘 업데이트된 보고서가 없습니다.
000480 오늘 업데이트된 보고서가 없습니다.
000490 오늘 업데이트된 보고서가 없습니다.
000500 오늘 업데이트된 보고서가 없습니다.
000520 오늘 업데이트된 보고서가 없습니다.
000590 오늘 업데이트된 보고서가 없습니다.
000640 오늘 업데이트된 보고서가 없습니다.
000650 오늘 업데이트된 보고서가 없습니다.
000660 오늘 업데이트된 보고서가 없습니다.
000670 오늘 업데이트된 보고서가 없습니다.
000680 오늘 업데이트된 보고서가 없습니다.
000700 오늘 업데이트된 보고서가 없습니다.
000720 오늘 업데이트된 보고서가 없습니다.
000760 오늘 업데이트된 보고서가 없습니다.
000850 오늘 업데이트된 보고서가 없습니다.
000860 오늘 업데이트된 보고서가 없습니다.
000880 오늘 업데이트된 보고서가 없습니다.
0

020180 오늘 업데이트된 보고서가 없습니다.
020400 오늘 업데이트된 보고서가 없습니다.
020560 오늘 업데이트된 보고서가 없습니다.
020710 오늘 업데이트된 보고서가 없습니다.
020760 오늘 업데이트된 보고서가 없습니다.
021040 오늘 업데이트된 보고서가 없습니다.
021050 오늘 업데이트된 보고서가 없습니다.
021080 오늘 업데이트된 보고서가 없습니다.
021240 오늘 업데이트된 보고서가 없습니다.
021320 오늘 업데이트된 보고서가 없습니다.
021650 오늘 업데이트된 보고서가 없습니다.
021820 오늘 업데이트된 보고서가 없습니다.
022100 오늘 업데이트된 보고서가 없습니다.
022220 오늘 업데이트된 보고서가 없습니다.
023000 오늘 업데이트된 보고서가 없습니다.
023150 오늘 업데이트된 보고서가 없습니다.
023160 오늘 업데이트된 보고서가 없습니다.
023350 오늘 업데이트된 보고서가 없습니다.
023410 오늘 업데이트된 보고서가 없습니다.
023440 오늘 업데이트된 보고서가 없습니다.
023450 오늘 업데이트된 보고서가 없습니다.
023460 오늘 업데이트된 보고서가 없습니다.
023530 오늘 업데이트된 보고서가 없습니다.
023590 오늘 업데이트된 보고서가 없습니다.
023600 오늘 업데이트된 보고서가 없습니다.
023770 오늘 업데이트된 보고서가 없습니다.
023790 오늘 업데이트된 보고서가 없습니다.
023800 오늘 업데이트된 보고서가 없습니다.
023810 오늘 업데이트된 보고서가 없습니다.
023900 오늘 업데이트된 보고서가 없습니다.
023910 오늘 업데이트된 보고서가 없습니다.
023960 오늘 업데이트된 보고서가 없습니다.
024060 오늘 업데이트된 보고서가 없습니다.
024070 오늘 업데이트된 보고서가 없습니다.
024090 오늘 업데이트된 보고서가 없습니다.
024120 오늘 업데이트된 보고서가 없습니다.
024720 오늘 업데이트된 보고서가 없습니다.
0

046120 오늘 업데이트된 보고서가 없습니다.
046140 오늘 업데이트된 보고서가 없습니다.
046210 오늘 업데이트된 보고서가 없습니다.
046310 오늘 업데이트된 보고서가 없습니다.
046390 오늘 업데이트된 보고서가 없습니다.
046440 오늘 업데이트된 보고서가 없습니다.
046890 오늘 업데이트된 보고서가 없습니다.
046940 오늘 업데이트된 보고서가 없습니다.
046970 오늘 업데이트된 보고서가 없습니다.
047040 오늘 업데이트된 보고서가 없습니다.
047050 오늘 업데이트된 보고서가 없습니다.
047080 오늘 업데이트된 보고서가 없습니다.
047310 오늘 업데이트된 보고서가 없습니다.
047400 오늘 업데이트된 보고서가 없습니다.
047560 오늘 업데이트된 보고서가 없습니다.
047770 오늘 업데이트된 보고서가 없습니다.
047810 오늘 업데이트된 보고서가 없습니다.
047820 오늘 업데이트된 보고서가 없습니다.
047920 오늘 업데이트된 보고서가 없습니다.
048260 오늘 업데이트된 보고서가 없습니다.
048410 오늘 업데이트된 보고서가 없습니다.
048430 오늘 업데이트된 보고서가 없습니다.
048470 오늘 업데이트된 보고서가 없습니다.
048530 오늘 업데이트된 보고서가 없습니다.
048550 오늘 업데이트된 보고서가 없습니다.
048770 오늘 업데이트된 보고서가 없습니다.
048830 오늘 업데이트된 보고서가 없습니다.
048870 오늘 업데이트된 보고서가 없습니다.
048910 오늘 업데이트된 보고서가 없습니다.
049070 오늘 업데이트된 보고서가 없습니다.
049080 오늘 업데이트된 보고서가 없습니다.
049120 오늘 업데이트된 보고서가 없습니다.
049180 오늘 업데이트된 보고서가 없습니다.
049430 오늘 업데이트된 보고서가 없습니다.
049470 오늘 업데이트된 보고서가 없습니다.
049480 오늘 업데이트된 보고서가 없습니다.
049520 오늘 업데이트된 보고서가 없습니다.
0

095190 오늘 업데이트된 보고서가 없습니다.
095270 오늘 업데이트된 보고서가 없습니다.
095340 오늘 업데이트된 보고서가 없습니다.
095500 오늘 업데이트된 보고서가 없습니다.
095570 오늘 업데이트된 보고서가 없습니다.
095610 오늘 업데이트된 보고서가 없습니다.
095660 오늘 업데이트된 보고서가 없습니다.
095700 오늘 업데이트된 보고서가 없습니다.
095720 오늘 업데이트된 보고서가 없습니다.
095910 오늘 업데이트된 보고서가 없습니다.
096040 오늘 업데이트된 보고서가 없습니다.
096240 오늘 업데이트된 보고서가 없습니다.
096350 오늘 업데이트된 보고서가 없습니다.
096530 오늘 업데이트된 보고서가 없습니다.
096610 오늘 업데이트된 보고서가 없습니다.
096630 오늘 업데이트된 보고서가 없습니다.
096640 오늘 업데이트된 보고서가 없습니다.
096690 오늘 업데이트된 보고서가 없습니다.
096760 오늘 업데이트된 보고서가 없습니다.
096770 오늘 업데이트된 보고서가 없습니다.
096870 오늘 업데이트된 보고서가 없습니다.
097230 오늘 업데이트된 보고서가 없습니다.
097520 오늘 업데이트된 보고서가 없습니다.
097780 오늘 업데이트된 보고서가 없습니다.
097800 오늘 업데이트된 보고서가 없습니다.
097870 오늘 업데이트된 보고서가 없습니다.
097950 오늘 업데이트된 보고서가 없습니다.
098120 오늘 업데이트된 보고서가 없습니다.
098460 오늘 업데이트된 보고서가 없습니다.
098660 오늘 업데이트된 보고서가 없습니다.
099190 오늘 업데이트된 보고서가 없습니다.
099220 오늘 업데이트된 보고서가 없습니다.
099320 오늘 업데이트된 보고서가 없습니다.
099410 오늘 업데이트된 보고서가 없습니다.
099440 오늘 업데이트된 보고서가 없습니다.
099520 오늘 업데이트된 보고서가 없습니다.
099750 오늘 업데이트된 보고서가 없습니다.
1

214450 오늘 업데이트된 보고서가 없습니다.
214610 오늘 업데이트된 보고서가 없습니다.
214680 오늘 업데이트된 보고서가 없습니다.
214870 오늘 업데이트된 보고서가 없습니다.
215000 오늘 업데이트된 보고서가 없습니다.
215090 오늘 업데이트된 보고서가 없습니다.
215100 오늘 업데이트된 보고서가 없습니다.
215200 오늘 업데이트된 보고서가 없습니다.
215360 오늘 업데이트된 보고서가 없습니다.
215380 오늘 업데이트된 보고서가 없습니다.
215480 오늘 업데이트된 보고서가 없습니다.
215600 오늘 업데이트된 보고서가 없습니다.
215790 오늘 업데이트된 보고서가 없습니다.
216050 오늘 업데이트된 보고서가 없습니다.
216080 오늘 업데이트된 보고서가 없습니다.
217190 오늘 업데이트된 보고서가 없습니다.
217270 오늘 업데이트된 보고서가 없습니다.
217330 오늘 업데이트된 보고서가 없습니다.
217480 오늘 업데이트된 보고서가 없습니다.
217500 오늘 업데이트된 보고서가 없습니다.
217600 오늘 업데이트된 보고서가 없습니다.
217620 오늘 업데이트된 보고서가 없습니다.
217730 오늘 업데이트된 보고서가 없습니다.
217820 오늘 업데이트된 보고서가 없습니다.
218150 오늘 업데이트된 보고서가 없습니다.
218410 오늘 업데이트된 보고서가 없습니다.
219130 오늘 업데이트된 보고서가 없습니다.
219420 오늘 업데이트된 보고서가 없습니다.
219550 오늘 업데이트된 보고서가 없습니다.
219750 오늘 업데이트된 보고서가 없습니다.
220100 오늘 업데이트된 보고서가 없습니다.
220180 오늘 업데이트된 보고서가 없습니다.
220260 오늘 업데이트된 보고서가 없습니다.
220630 오늘 업데이트된 보고서가 없습니다.
221610 오늘 업데이트된 보고서가 없습니다.
221840 오늘 업데이트된 보고서가 없습니다.
221980 오늘 업데이트된 보고서가 없습니다.
2

In [15]:
def db_cash_flow_update():
    path = os.path.join("dart", "dart")
    companies = os.listdir(path)
    starting_time = datetime.datetime.now()
    for corp_code in companies:
        # last_modified ~ print("----")는 일단 전체 DB 업데이트 한번 다 한 이후에 실행할 것!
        
        # 해당 폴더 마지막으로 업데이트 된 시각
        last_modified = convert_datetime(time.ctime(os.path.getmtime(os.path.join("dart", "dart", corp_code, "financial_statements", "cash_flow"))))
        # 현재 시각
        date_today = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S').split(" ")[0]
    
        if time_comparison(last_modified, date_today): # 오늘 추가된 새로운 파일이 없을 시에는 표준계정에 업데이트 안하고 넘어감.
            print(corp_code, "오늘 업데이트된 보고서가 없습니다.")
            continue
        else:
            print(corp_code, datetime.datetime.now(), "에 표준계정 업데이트 시작..." )
            update_cash_flow(corp_code)
            print(corp_code, datetime.datetime.now(), "에 표준계정 업데이트 완료!")
            print("-------------------------------------------------------------------------------------")
            
            
    end_time = datetime.datetime.now()
    print("전체 현금흐름표 DB화 작업 완료!")
    print("시작 시간: ", starting_time, "종료 시간: ", end_time)
db_cash_flow_update()

000020 오늘 업데이트된 보고서가 없습니다.
000040 오늘 업데이트된 보고서가 없습니다.
000050 오늘 업데이트된 보고서가 없습니다.
000070 오늘 업데이트된 보고서가 없습니다.
000080 오늘 업데이트된 보고서가 없습니다.
000100 오늘 업데이트된 보고서가 없습니다.
000120 오늘 업데이트된 보고서가 없습니다.
000140 오늘 업데이트된 보고서가 없습니다.
000150 오늘 업데이트된 보고서가 없습니다.
000180 오늘 업데이트된 보고서가 없습니다.
000210 오늘 업데이트된 보고서가 없습니다.
000220 오늘 업데이트된 보고서가 없습니다.
000230 오늘 업데이트된 보고서가 없습니다.
000240 오늘 업데이트된 보고서가 없습니다.
000250 오늘 업데이트된 보고서가 없습니다.
000270 오늘 업데이트된 보고서가 없습니다.
000300 오늘 업데이트된 보고서가 없습니다.
000320 오늘 업데이트된 보고서가 없습니다.
000390 오늘 업데이트된 보고서가 없습니다.
000430 오늘 업데이트된 보고서가 없습니다.
000440 오늘 업데이트된 보고서가 없습니다.
000480 오늘 업데이트된 보고서가 없습니다.
000490 오늘 업데이트된 보고서가 없습니다.
000500 오늘 업데이트된 보고서가 없습니다.
000520 오늘 업데이트된 보고서가 없습니다.
000590 오늘 업데이트된 보고서가 없습니다.
000640 오늘 업데이트된 보고서가 없습니다.
000650 오늘 업데이트된 보고서가 없습니다.
000660 오늘 업데이트된 보고서가 없습니다.
000670 오늘 업데이트된 보고서가 없습니다.
000680 오늘 업데이트된 보고서가 없습니다.
000700 오늘 업데이트된 보고서가 없습니다.
000720 오늘 업데이트된 보고서가 없습니다.
000760 오늘 업데이트된 보고서가 없습니다.
000850 오늘 업데이트된 보고서가 없습니다.
000860 오늘 업데이트된 보고서가 없습니다.
000880 오늘 업데이트된 보고서가 없습니다.
0

010040 오늘 업데이트된 보고서가 없습니다.
010060 오늘 업데이트된 보고서가 없습니다.
010100 오늘 업데이트된 보고서가 없습니다.
010120 오늘 업데이트된 보고서가 없습니다.
010130 오늘 업데이트된 보고서가 없습니다.
010140 오늘 업데이트된 보고서가 없습니다.
010170 오늘 업데이트된 보고서가 없습니다.
010240 오늘 업데이트된 보고서가 없습니다.
010280 오늘 업데이트된 보고서가 없습니다.
010400 오늘 업데이트된 보고서가 없습니다.
010420 오늘 업데이트된 보고서가 없습니다.
010470 오늘 업데이트된 보고서가 없습니다.
010580 오늘 업데이트된 보고서가 없습니다.
010600 오늘 업데이트된 보고서가 없습니다.
010620 오늘 업데이트된 보고서가 없습니다.
010640 오늘 업데이트된 보고서가 없습니다.
010660 오늘 업데이트된 보고서가 없습니다.
010690 오늘 업데이트된 보고서가 없습니다.
010770 오늘 업데이트된 보고서가 없습니다.
010780 오늘 업데이트된 보고서가 없습니다.
010820 오늘 업데이트된 보고서가 없습니다.
010950 오늘 업데이트된 보고서가 없습니다.
010960 오늘 업데이트된 보고서가 없습니다.
011000 오늘 업데이트된 보고서가 없습니다.
011040 오늘 업데이트된 보고서가 없습니다.
011070 오늘 업데이트된 보고서가 없습니다.
011080 오늘 업데이트된 보고서가 없습니다.
011090 오늘 업데이트된 보고서가 없습니다.
011150 오늘 업데이트된 보고서가 없습니다.
011170 오늘 업데이트된 보고서가 없습니다.
011200 오늘 업데이트된 보고서가 없습니다.
011210 오늘 업데이트된 보고서가 없습니다.
011230 오늘 업데이트된 보고서가 없습니다.
011280 오늘 업데이트된 보고서가 없습니다.
011300 오늘 업데이트된 보고서가 없습니다.
011320 오늘 업데이트된 보고서가 없습니다.
011330 오늘 업데이트된 보고서가 없습니다.
0

035250 오늘 업데이트된 보고서가 없습니다.
035290 오늘 업데이트된 보고서가 없습니다.
035420 오늘 업데이트된 보고서가 없습니다.
035460 오늘 업데이트된 보고서가 없습니다.
035510 오늘 업데이트된 보고서가 없습니다.
035600 오늘 업데이트된 보고서가 없습니다.
035610 오늘 업데이트된 보고서가 없습니다.
035620 오늘 업데이트된 보고서가 없습니다.
035720 오늘 업데이트된 보고서가 없습니다.
035760 오늘 업데이트된 보고서가 없습니다.
035810 오늘 업데이트된 보고서가 없습니다.
035890 오늘 업데이트된 보고서가 없습니다.
035900 오늘 업데이트된 보고서가 없습니다.
036000 오늘 업데이트된 보고서가 없습니다.
036010 오늘 업데이트된 보고서가 없습니다.
036030 오늘 업데이트된 보고서가 없습니다.
036090 오늘 업데이트된 보고서가 없습니다.
036120 오늘 업데이트된 보고서가 없습니다.
036170 오늘 업데이트된 보고서가 없습니다.
036180 오늘 업데이트된 보고서가 없습니다.
036190 오늘 업데이트된 보고서가 없습니다.
036200 오늘 업데이트된 보고서가 없습니다.
036420 오늘 업데이트된 보고서가 없습니다.
036460 오늘 업데이트된 보고서가 없습니다.
036480 오늘 업데이트된 보고서가 없습니다.
036490 오늘 업데이트된 보고서가 없습니다.
036530 오늘 업데이트된 보고서가 없습니다.
036540 오늘 업데이트된 보고서가 없습니다.
036560 오늘 업데이트된 보고서가 없습니다.
036570 오늘 업데이트된 보고서가 없습니다.
036580 오늘 업데이트된 보고서가 없습니다.
036620 오늘 업데이트된 보고서가 없습니다.
036630 오늘 업데이트된 보고서가 없습니다.
036640 오늘 업데이트된 보고서가 없습니다.
036670 오늘 업데이트된 보고서가 없습니다.
036690 오늘 업데이트된 보고서가 없습니다.
036710 오늘 업데이트된 보고서가 없습니다.
0

059100 오늘 업데이트된 보고서가 없습니다.
059120 오늘 업데이트된 보고서가 없습니다.
059210 오늘 업데이트된 보고서가 없습니다.
060150 오늘 업데이트된 보고서가 없습니다.
060230 오늘 업데이트된 보고서가 없습니다.
060240 오늘 업데이트된 보고서가 없습니다.
060250 오늘 업데이트된 보고서가 없습니다.
060260 오늘 업데이트된 보고서가 없습니다.
060280 오늘 업데이트된 보고서가 없습니다.
060300 오늘 업데이트된 보고서가 없습니다.
060310 오늘 업데이트된 보고서가 없습니다.
060370 오늘 업데이트된 보고서가 없습니다.
060380 오늘 업데이트된 보고서가 없습니다.
060480 오늘 업데이트된 보고서가 없습니다.
060540 오늘 업데이트된 보고서가 없습니다.
060560 오늘 업데이트된 보고서가 없습니다.
060570 오늘 업데이트된 보고서가 없습니다.
060590 오늘 업데이트된 보고서가 없습니다.
060720 오늘 업데이트된 보고서가 없습니다.
060850 오늘 업데이트된 보고서가 없습니다.
060900 오늘 업데이트된 보고서가 없습니다.
060980 오늘 업데이트된 보고서가 없습니다.
061040 오늘 업데이트된 보고서가 없습니다.
061250 오늘 업데이트된 보고서가 없습니다.
061970 오늘 업데이트된 보고서가 없습니다.
062860 오늘 업데이트된 보고서가 없습니다.
062970 오늘 업데이트된 보고서가 없습니다.
063080 오늘 업데이트된 보고서가 없습니다.
063160 오늘 업데이트된 보고서가 없습니다.
063170 오늘 업데이트된 보고서가 없습니다.
063440 오늘 업데이트된 보고서가 없습니다.
063570 오늘 업데이트된 보고서가 없습니다.
063760 오늘 업데이트된 보고서가 없습니다.
064090 오늘 업데이트된 보고서가 없습니다.
064240 오늘 업데이트된 보고서가 없습니다.
064260 오늘 업데이트된 보고서가 없습니다.
064290 오늘 업데이트된 보고서가 없습니다.
0

097230 오늘 업데이트된 보고서가 없습니다.
097520 오늘 업데이트된 보고서가 없습니다.
097780 오늘 업데이트된 보고서가 없습니다.
097800 오늘 업데이트된 보고서가 없습니다.
097870 오늘 업데이트된 보고서가 없습니다.
097950 오늘 업데이트된 보고서가 없습니다.
098120 오늘 업데이트된 보고서가 없습니다.
098460 오늘 업데이트된 보고서가 없습니다.
098660 오늘 업데이트된 보고서가 없습니다.
099190 오늘 업데이트된 보고서가 없습니다.
099220 오늘 업데이트된 보고서가 없습니다.
099320 오늘 업데이트된 보고서가 없습니다.
099410 오늘 업데이트된 보고서가 없습니다.
099440 오늘 업데이트된 보고서가 없습니다.
099520 오늘 업데이트된 보고서가 없습니다.
099750 오늘 업데이트된 보고서가 없습니다.
100030 오늘 업데이트된 보고서가 없습니다.
100090 오늘 업데이트된 보고서가 없습니다.
100120 오늘 업데이트된 보고서가 없습니다.
100130 오늘 업데이트된 보고서가 없습니다.
100220 오늘 업데이트된 보고서가 없습니다.
100250 오늘 업데이트된 보고서가 없습니다.
100590 오늘 업데이트된 보고서가 없습니다.
100660 오늘 업데이트된 보고서가 없습니다.
100700 오늘 업데이트된 보고서가 없습니다.
100840 오늘 업데이트된 보고서가 없습니다.
101000 오늘 업데이트된 보고서가 없습니다.
101060 오늘 업데이트된 보고서가 없습니다.
101140 오늘 업데이트된 보고서가 없습니다.
101160 오늘 업데이트된 보고서가 없습니다.
101170 오늘 업데이트된 보고서가 없습니다.
101240 오늘 업데이트된 보고서가 없습니다.
101330 오늘 업데이트된 보고서가 없습니다.
101360 오늘 업데이트된 보고서가 없습니다.
101390 오늘 업데이트된 보고서가 없습니다.
101400 오늘 업데이트된 보고서가 없습니다.
101490 오늘 업데이트된 보고서가 없습니다.
1

205500 오늘 업데이트된 보고서가 없습니다.
206400 오늘 업데이트된 보고서가 없습니다.
206560 오늘 업데이트된 보고서가 없습니다.
206640 오늘 업데이트된 보고서가 없습니다.
206650 오늘 업데이트된 보고서가 없습니다.
207760 오늘 업데이트된 보고서가 없습니다.
207940 오늘 업데이트된 보고서가 없습니다.
208140 오늘 업데이트된 보고서가 없습니다.
208340 오늘 업데이트된 보고서가 없습니다.
208350 오늘 업데이트된 보고서가 없습니다.
208370 오늘 업데이트된 보고서가 없습니다.
208640 오늘 업데이트된 보고서가 없습니다.
208710 오늘 업데이트된 보고서가 없습니다.
208860 오늘 업데이트된 보고서가 없습니다.
210540 오늘 업데이트된 보고서가 없습니다.
210980 오늘 업데이트된 보고서가 없습니다.
211270 오늘 업데이트된 보고서가 없습니다.
212560 오늘 업데이트된 보고서가 없습니다.
213090 오늘 업데이트된 보고서가 없습니다.
213420 오늘 업데이트된 보고서가 없습니다.
213500 오늘 업데이트된 보고서가 없습니다.
214150 오늘 업데이트된 보고서가 없습니다.
214180 오늘 업데이트된 보고서가 없습니다.
214260 오늘 업데이트된 보고서가 없습니다.
214270 오늘 업데이트된 보고서가 없습니다.
214310 오늘 업데이트된 보고서가 없습니다.
214320 오늘 업데이트된 보고서가 없습니다.
214330 오늘 업데이트된 보고서가 없습니다.
214370 오늘 업데이트된 보고서가 없습니다.
214390 오늘 업데이트된 보고서가 없습니다.
214420 오늘 업데이트된 보고서가 없습니다.
214430 오늘 업데이트된 보고서가 없습니다.
214450 오늘 업데이트된 보고서가 없습니다.
214610 오늘 업데이트된 보고서가 없습니다.
214680 오늘 업데이트된 보고서가 없습니다.
214870 오늘 업데이트된 보고서가 없습니다.
215000 오늘 업데이트된 보고서가 없습니다.
2

314930 오늘 업데이트된 보고서가 없습니다.
317120 오늘 업데이트된 보고서가 없습니다.
317240 오늘 업데이트된 보고서가 없습니다.
317330 오늘 업데이트된 보고서가 없습니다.
317400 오늘 업데이트된 보고서가 없습니다.
317530 오늘 업데이트된 보고서가 없습니다.
317690 오늘 업데이트된 보고서가 없습니다.
317770 오늘 업데이트된 보고서가 없습니다.
317830 오늘 업데이트된 보고서가 없습니다.
317850 오늘 업데이트된 보고서가 없습니다.
317870 오늘 업데이트된 보고서가 없습니다.
318000 오늘 업데이트된 보고서가 없습니다.
318010 오늘 업데이트된 보고서가 없습니다.
318020 오늘 업데이트된 보고서가 없습니다.
318410 오늘 업데이트된 보고서가 없습니다.
319400 오늘 업데이트된 보고서가 없습니다.
319660 오늘 업데이트된 보고서가 없습니다.
320000 오늘 업데이트된 보고서가 없습니다.
321550 오늘 업데이트된 보고서가 없습니다.
321820 오늘 업데이트된 보고서가 없습니다.
322000 오늘 업데이트된 보고서가 없습니다.
322180 오늘 업데이트된 보고서가 없습니다.
322310 오늘 업데이트된 보고서가 없습니다.
322510 오늘 업데이트된 보고서가 없습니다.
322780 오늘 업데이트된 보고서가 없습니다.
323230 오늘 업데이트된 보고서가 없습니다.
323990 오늘 업데이트된 보고서가 없습니다.
326030 오늘 업데이트된 보고서가 없습니다.
327260 오늘 업데이트된 보고서가 없습니다.
330350 오늘 업데이트된 보고서가 없습니다.
330590 오늘 업데이트된 보고서가 없습니다.
330860 오늘 업데이트된 보고서가 없습니다.
331920 오늘 업데이트된 보고서가 없습니다.
332370 오늘 업데이트된 보고서가 없습니다.
332570 오늘 업데이트된 보고서가 없습니다.
332710 오늘 업데이트된 보고서가 없습니다.
333620 오늘 업데이트된 보고서가 없습니다.
3